## Write DYFI geocodings to db

This notebook demonstrates how to write geocoded locations for DYFI enquiries stored in a CSV file to the database.

CSV files need to have at least the following columns: 'id_web', 'longitude', 'latitude', 'address_type'

Note that the CSV file must be on the same computer as the notebook server!

If it is on a remote server, then the CSV file must be readable to the user under whose name the server is running!

Import python modules

In [46]:
import os
import csv

import db.simpledb as simpledb
from secrets.seismodb import (host, database, user_rw, passwd_rw)

from eqcatalog.rob.seismodb import query_seismodb_table

Define CSV file to work with

In [47]:
## On Windows PC
csv_folder = "D:\\Earthquake Reports\\20190623\\DYFI geocoding"
## On poseidon
#csv_folder = "/home/koenvn/"
csv_filename = "macroseismic_inq_for_8285_corrected_geocoded-ArcGIS.csv"
csv_file = os.path.join(csv_folder, csv_filename)

In [48]:
def read_geocoded_records_from_db():
    """
    Read all id_web IDs currently in web_location database table
    
    :return:
        list of dicts, mapping column names ('id_web', 'quality') to values
    """
    table_name = 'web_location'
    column_clause = ['id_web', 'quality']
    db_recs = query_seismodb_table(table_name, column_clause=column_clause)
    
    print('Read %d geocoded records from database' % len(db_recs))
    
    return db_recs

In [49]:
db_recs = read_geocoded_records_from_db()

Read 25855 geocoded records from database


In [50]:
def read_geocodings_from_csv(csv_file):
    """
    Read geocoded results from CSV file
    
    Note: 'address_type' (str) will be converted to quality (int)
    
    :param csv_file:
        str, full path to CSV file (must be on same computer as notebook server!)
    
    :return:
        list of dicts mapping column names ('id_web', 'lon', 'lat', 'quality')
        to values
    """
    recs = []
    
    with open(csv_file) as csvf:
        reader = csv.DictReader(csvf)
        for row in reader:
            rec = {}
            rec['id_web'] = int(row['id_web'])
            rec['longitude'] = float(row['Geo_Longitude'])
            rec['latitude'] = float(row['Geo_Latitude'])
            address_type = row['address type']
            
            ## Convert ArcGIS address_type to location quality

            ## Geocoder confidence:
            ## 10 : 250 m
            ##  9 : 500 m
            ##  8 : 1 km
            ##  7 : 5 km
            ##  6 : 7.5 km
            ##  5 : 10 km
            ##  4 : 15 km
            ##  3 : 20 km
            ##  2 : 25 km
            ##  1 : > 25 km
            ##  0 : undetermined

            ## Google API v2 GGeoAddressAccuracy:
            ## Constant | Description
            ## 0 Unknown location.
            ## 1 Country level accuracy.
            ## 2 Region (state, province, prefecture, etc.) level accuracy.
            ## 3 Sub-region (county, municipality, etc.) level accuracy.
            ## 4 Town (city, village) level accuracy.
            ## 5 Post code (zip code) level accuracy.
            ## 6 Street level accuracy.
            ## 7 Intersection level accuracy.
            ## 8 Address level accuracy.
            ## 9 Premise (building name, property name, shopping center, etc.) level accuracy.

            """
            rec['confidence'] = {'SubAddress': 10,
                                'PointAddress': 10,
                                'StreetAddress': 10,
                                'StreetInt': 9,
                                'StreetAddressExt': 9,
                                'DistanceMarker': 9,
                                'StreetName': 8,
                                'Locality': 8,
                                'PostalLoc': 7,
                                'PostalExt': 7,
                                'Postal': 7,
                                'POI': 7}.get(address_type, 0)
            """

            rec['quality'] = {'SubAddress': 9,
                                'PointAddress': 9,
                                'StreetAddress': 8,
                                'StreetInt': 7,
                                'StreetAddressExt': 7,
                                'DistanceMarker': 7,
                                'StreetName': 6,
                                'Locality': 5,
                                'PostalLoc': 5,
                                'PostalExt': 5,
                                'Postal': 4,
                                'POI': 4}.get(address_type, 0)
            
            recs.append(rec)
    
    print('Read %d geocoded records from CSV file' % len(recs))
    
    return recs

In [51]:
csv_recs = read_geocodings_from_csv(csv_file)

Read 37 geocoded records from CSV file


In [52]:
def determine_recs_to_write_to_db(csv_recs):
    """
    Determine which CSV geocodings need to be written to the database
    
    :param csv_recs:
        list of dicts returned by :func:`read_geocodings_from_csv`
    
    :return:
        (recs_to_add, recs_to_modify) tuple of lists of dicts
    """
    recs_to_add, recs_to_modify = [], []

    db_recs = read_geocoded_records_from_db()
    web_ids = [rec['id_web'] for rec in db_recs]

    num_db_recs = 0
    for rec in csv_recs:
        id_web = rec['id_web']
        lon = rec['longitude']
        lat = rec['latitude']

        try:
            r = web_ids.index(id_web)
        except:
            recs_to_add.append(rec)
        else:
            ## Only overwrite existing locations if quality is better
            if rec['quality'] > db_recs[r]['quality']:
                recs_to_modify.append(rec)
            else:
                num_db_recs += 1
    
    print('Identified %d new records to add / %d records to modify'
         % (len(recs_to_add), len(recs_to_modify)))
    print('%d records in db have higher or equal location quality' % num_db_recs)

    return (recs_to_add, recs_to_modify)

In [53]:
recs_to_add, recs_to_modify = determine_recs_to_write_to_db(csv_recs)

Read 25855 geocoded records from database
Identified 0 new records to add / 0 records to modify
37 records in db have higher or equal location quality


In [54]:
def write_csv_geocodings_to_db(csv_recs, dry_run=True):
    """
    Write CSV geocodings to database
    
    :param csv_recs:
        list of dicts returned by :func:`read_geocodings_from_csv`
    :param dry_run:
        bool, whether to really write to the database (False)
        or just print how many records will be added or updated (True)
        Note: it is not possible to undo this operation!
    """
    recs_to_add, recs_to_modify = determine_recs_to_write_to_db(csv_recs)

    seismodb = simpledb.MySQLDB(database, host, user_rw, passwd_rw)
    table_name = 'web_location'

    if len(recs_to_add):
        print("Adding %d new records" % len(recs_to_add))
        if not dry_run:
            seismodb.add_records(table_name, recs_to_add, dry_run=True)

    if len(recs_to_modify):
        print("Updating %d existing records" % len(recs_to_modify))
        if not dry_run:
            seismodb.update_rows(table_name, recs_to_modify, 'id_web', dry_run=True)


In [55]:
write_csv_geocodings_to_db(csv_recs, dry_run=True)

Read 25855 geocoded records from database
Identified 0 new records to add / 0 records to modify
37 records in db have higher or equal location quality
